# Import Dataset

In [1]:
import os
import pickle
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from collections import Counter
from tqdm import tqdm

import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix

data_dir = "./dataset"
%cd -q dataset

files = os.listdir()
print("View all datasets:")
for f in files:
    print(f)

View all datasets:
disc_df.csv
bda2024_202203-202402_討論數據_dcard.csv
.DS_Store
bda2024_202203-202402_內容數據_新聞3-8.csv
bda2024_202203-202402_內容數據_新聞3-9.csv
bda2024_202203-202402_內容數據_新聞1-9.csv
bda2024_202203-202402_內容數據_新聞1-8.csv
bda2024_微股力_社群PKTD-2年-3.csv
bda2024_微股力_籌碼數據-2年.csv
bda2024_微股力_社群PKTD-2年-2.csv
bda2024_微股力_社群PKTD-2年-0.csv
bda2024_微股力_社群PKTD-2年-1.csv
bda2024_微股力_社群PKTD-2年-5.csv
bda2024_202203-202402_討論數據_ptt.csv
bda2024_微股力_社群PKTD-2年-4.csv
bda2024_微股力_社群PKTD-2年-6.csv
bda2024_202203-202402_內容數據_新聞2-8.csv
bda2024_微股力_財報數據-2年.csv
bda2024_202203-202402_內容數據_新聞2-9.csv
bda2024_微股力_社群PKTD-2年-7.csv
bda2024_202203-202402_內容數據_新聞2-4.csv
bda2024_202203-202402_內容數據_新聞2-5.csv
bda2024_微股力_社群PKTD-2年-9.csv
bda2024_202203-202402_內容數據_新聞2-7.csv
bda2024_202203-202402_內容數據_新聞2-6.csv
bda2024_微股力_社群PKTD-2年-8.csv
bda2024_202203-202402_內容數據_新聞2-2.csv
bda2024_202203-202402_內容數據_新聞2-3.csv
bda2024_202203-202402_內容數據_新聞2-1.csv
bda2024_微股力_個股交易數據-2年.csv
bda2024_202203-202402_內容數據_新聞2-0.csv
bda2024_202203-

/opt/homebrew/anaconda3/envs/CV2023/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Data Preprocess

In [2]:
def load_df(filepath, preview=True):
    print(f"\n----- Loading {filepath}... -----")
    df = pd.read_csv(filepath)
    print(f"Size of dataframe: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    if preview:
        print(df.head())
    return df

因為要討論 ppt / 各種論壇的情緒指標和討論聲量和產業的關係，因此沒有用新聞內容的 data


In [3]:
disc_dcard_df = load_df("bda2024_202203-202402_討論數據_dcard.csv", preview=False)
disc_dcard_df.rename(columns={'forum': 'p_type'}, inplace=True)    # Repair column name typo in data
disc_m1_df = load_df("bda2024_202203-202402_討論數據_mobile01-1.csv", preview=False)
disc_m2_df = load_df("bda2024_202203-202402_討論數據_mobile01-2.csv", preview=False)
disc_ptt_df = load_df("bda2024_202203-202402_討論數據_ptt.csv", preview=False)
disc_df = pd.concat([disc_dcard_df, disc_m1_df, disc_m2_df, disc_ptt_df], ignore_index=True)

transaction_df = load_df("bda2024_微股力_個股交易數據-2年.csv")

transaction_df['stock_symbol'] = transaction_df['stock_symbol'].astype(str)


----- Loading bda2024_202203-202402_討論數據_dcard.csv... -----
Size of dataframe: (231320, 10)
Columns: ['id', 'forum', 's_name', 's_area_name', 'post_time', 'title', 'author', 'content', 'page_url', 'content_type']

----- Loading bda2024_202203-202402_討論數據_mobile01-1.csv... -----
Size of dataframe: (48725, 10)
Columns: ['id', 'p_type', 's_name', 's_area_name', 'post_time', 'title', 'author', 'content', 'page_url', 'content_type']

----- Loading bda2024_202203-202402_討論數據_mobile01-2.csv... -----
Size of dataframe: (157939, 10)
Columns: ['id', 'p_type', 's_name', 's_area_name', 'post_time', 'title', 'author', 'content', 'page_url', 'content_type']

----- Loading bda2024_202203-202402_討論數據_ptt.csv... -----
Size of dataframe: (50805, 9)
Columns: ['id', 'p_type', 's_name', 's_area_name', 'post_time', 'title', 'author', 'content', 'page_url']

----- Loading bda2024_微股力_個股交易數據-2年.csv... -----
Size of dataframe: (1154225, 8)
Columns: ['stock_name', 'stock_symbol', 'open', 'high', 'low', 'close'

/var/folders/7g/rxqxvrmd0z1bnnny3trbtfz80000gn/T/ipykernel_36421/550884886.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


In [4]:
disc_df = disc_df[["id", "post_time", "content"]]

disc_df["content"] = disc_df["content"].astype(str)

# 股價漲跌 Labeling

AI 概念股：

https://www.sinotrade.com.tw/richclub/hotstock/-65af4cb1880d9e29902a677f

https://www.wantgoo.com/index/%5E435/stocks


利用「全部 AI 概念股 n 天後的股價」減掉「全部 AI 概念股今天的股價」來當作判斷，若漲幅超過 m% 則判斷為漲。

n = 7, m = 5 (暫定)


In [5]:
# get target stocks

target_stock_name_1 = ['廣達', '緯創', '台積電', '創意', '世芯-KY', '智原', '智邦', '信驊', '譜瑞-KY', '日月光投控', '台達電', '光寶科', '群光', '奇鋐', '金像電', '台燿', '嘉澤', '技嘉', '緯穎', '英業達', '鴻海', '聯發科', '聯茂', 'M31']
target_stock_name_2 = ['AMAX-KY','緯創','凌群','創意','東元','英業達','原相','長佳智能','精誠','鴻海','華碩','微星','金寶','聯發科','世芯-KY','京元電子','美律','亞信','研華','台積電','宏碁','走著瞧-創','鈺創','廣達','凌華','零壹','台達電','樺漢','群電']

#get rid of the duplicate stock names
target_stock_name = target_stock_name_1 + target_stock_name_2
target_stock_name = list(set(target_stock_name))

In [6]:
transaction_df['date'] = pd.to_datetime(transaction_df['date']).dt.date
disc_df['post_time'] = pd.to_datetime(disc_df['post_time'])
disc_df['post_time'] = disc_df['post_time'].dt.date

#add a column "rise" in transaction_df to indicate whether the stock price rises or not, default is null
disc_df["rise"] = np.nan

dates_list = sorted(list(set(transaction_df["date"])))

In [7]:
# if the sum of the stock price in stock_name n days later is higher than the sum of stock price today by more than 5%, then label today's disc_df["rise"] as 1
# if the sum of the stock price in stock_name n days later is lower than the sum of stock price today by more than 5%, then label today's disc_df["rise"] as -1
# if the sum of the stock price in stock_name n days later is within 5% of the sum of stock price today, then label today's disc_df["rise"] as 0

n = 7
m = 0.05

for i in range(len(dates_list[:-7])):
    #check the availability of stock price data of stock_name today
    stock_data_today = transaction_df[transaction_df["date"] == dates_list[i]]
    stock_data_today_name = stock_data_today["stock_name"].values

    #check the availability of stock price data of stock_name n days later
    stock_data_later = transaction_df[transaction_df["date"] == dates_list[i + n]]
    stock_data_later_name = stock_data_later["stock_name"].values

    #get the available stocks on both today and later
    stock_name = [stock for stock in stock_data_today_name if stock in stock_data_later_name]

    stock_name = [stock for stock in stock_name if stock in target_stock_name]
    
    if len(stock_name) == 0:
        continue

    #if stock_name is not available in stock data, then label today's disc_df["rise"] as -100

    #calculate the sum of stock price of stock_name today
    stock_price_today = 0
    for stock in stock_name:
        if stock in stock_data_today_name:
            stock_price_today += stock_data_today[stock_data_today["stock_name"] == stock]["open"].values[0]

    #calculate the sum of stock price of stock_name n days later
    stock_price_later = 0
    for stock in stock_name:
        if stock in stock_data_later_name:
            stock_price_later += stock_data_later[stock_data_later["stock_name"] == stock]["close"].values[0]

    #calculate the percentage change of stock price, and label the data in disc_df["rise"]
    percentage_change = (stock_price_later - stock_price_today) / stock_price_today
    if percentage_change > m:
        disc_df.loc[disc_df["post_time"] == dates_list[i], "rise"] = 1
    elif percentage_change < -m:
        disc_df.loc[disc_df["post_time"] == dates_list[i], "rise"] = -1
    else:
        disc_df.loc[disc_df["post_time"] == dates_list[i], "rise"] = 0

In [8]:
# get rid of the data with disc_df["rise"] == null
disc_df = disc_df[disc_df["rise"].notnull()]
# get rid of the data with disc_df["rise"] == 0
disc_df = disc_df[disc_df["rise"] != 0]

# reassign the post_id
disc_df["post_id"] = range(len(disc_df))

#export disc_df to csv
disc_df.to_csv("disc_df.csv", index=False)

# 找出具鑑別力 (扣除共通字詞) 的關鍵字列表，合起來建構向量空間

1000維度（？），把所有資料轉換成以這個向量空間為主的向量（嗎）

先取 5000 筆當作 training data 來建構具鑑別力 (扣除共通字詞) 的關 鍵字列表，合起來建構向量空間
1000 筆當作 testing data


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
# get 5000 of the data as training data
# get 1000 of the data as testing data

train_data, test_data = train_test_split(disc_df, train_size=5000, test_size=1000, random_state=42)

# Print the shapes of the training and testing sets
print("Training data shape:", train_data.shape)
print("Testing data shape:", test_data.shape)

Training data shape: (5000, 5)
Testing data shape: (1000, 5)


用 monpa 切(因為 CkipTagger 要 tensorflow 1.13， 但那要 python 3.8 垃圾)

(還是用 jieba 比較好？)

In [10]:
# use monpa to segment the content

# add a new column "segmented_content" to store the segmented content
train_data["segmented_content"] = np.nan
test_data["segmented_content"] = np.nan

tmp = []

import monpa
for i in range(len(train_data)):
    segmented_content = monpa.cut(train_data["content"].iloc[i])
    tmp.append(segmented_content)

train_data["segmented_content"] = tmp

tmp = []

import monpa
for i in range(len(test_data)):
    segmented_content = monpa.cut(test_data["content"].iloc[i])
    tmp.append(segmented_content)

test_data["segmented_content"] = tmp

+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
已找到 model檔。Found model file.


In [11]:
# export train_data and test_data to csv
train_data.to_csv("train_data.csv", index=False)
test_data.to_csv("test_data.csv", index=False)

In [12]:
# from collections import Counter
# # use all the text in the training data to build a dictionary
# dictionary = []
# for i in range(len(train_data)):
#     dictionary += train_data["segmented_content"].iloc[i]

# dictionary = list(set(dictionary))

# print("The size of the dictionary is:", len(dictionary))

# # for each data in training data, count the frequency of each word in the dictionary

